In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
from data_preprocessing import load_data, handle_outliers, preprocess_data
from model_building import build_model

In [3]:
def train_model(model, X_numerical, X_marca, X_modelo, X_versao, y, validation_split=0.2, epochs=100, batch_size=32):
    # Compile the model for regression
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Prepare categorical features for training
    X_marca = X_marca.reshape(-1, 1)
    X_modelo = X_modelo.reshape(-1, 1)
    X_versao = X_versao.reshape(-1, 1)

    # Train the model
    history = model.fit(
        [X_numerical, X_marca, X_modelo, X_versao],
        y,
        validation_split=validation_split,
        epochs=epochs,
        batch_size=batch_size
    )
    
    return history

In [4]:
# Load data
file_path = 'https://github.com/pedrogasparotti/challenges/raw/main/cars_train.csv'
df = load_data(file_path)

# Handle outliers
df = handle_outliers(df)

# Preprocess data
X_numerical, X_marca, X_modelo, X_versao, y = preprocess_data(df)

# Split the data
X_numerical_train, X_numerical_test, X_marca_train, X_marca_test, X_modelo_train, X_modelo_test, X_versao_train, X_versao_test, y_train, y_test = train_test_split(
    X_numerical, X_marca, X_modelo, X_versao, y, test_size=0.2, random_state=42
)

# Number of unique categories in each categorical feature
num_unique_categories = {
    'marca': df['marca'].nunique(),
    'modelo': df['modelo'].nunique(),
    'versao': df['versao'].nunique()
}

# Input shapes for numerical and categorical features
input_shape_numerical = (X_numerical.shape[1],)
input_shape_categorical = (1,)

# Build the model
embedding_size = 8
model = build_model(input_shape_numerical, input_shape_categorical, num_unique_categories, embedding_size)

# Train the model on the training data
history = train_model(model, X_numerical_train, X_marca_train, X_modelo_train, X_versao_train, y_train)

# Evaluate the model on the test data
mse = model.evaluate([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test], y_test)
print("Mean Squared Error on Test Data:", mse)

Epoch 1/100
583/583 [==============================] - 2s 2ms/step - loss: 12410201088.0000 - val_loss: 5094017024.0000
Epoch 2/100
583/583 [==============================] - 2s 3ms/step - loss: 4617553920.0000 - val_loss: 4005543680.0000
Epoch 3/100
583/583 [==============================] - 1s 2ms/step - loss: 4104091392.0000 - val_loss: 3799678976.0000
Epoch 4/100
583/583 [==============================] - 1s 2ms/step - loss: 3923952128.0000 - val_loss: 3623969536.0000
Epoch 5/100
583/583 [==============================] - 1s 2ms/step - loss: 3702711040.0000 - val_loss: 3373264128.0000
Epoch 6/100
583/583 [==============================] - 1s 2ms/step - loss: 3384036096.0000 - val_loss: 3021314816.0000
Epoch 7/100
583/583 [==============================] - 1s 2ms/step - loss: 2941949440.0000 - val_loss: 2566961408.0000
Epoch 8/100
583/583 [==============================] - 1s 2ms/step - loss: 2450670848.0000 - val_loss: 2173491456.0000
Epoch 9/100
583/583 [==========================

583/583 [==============================] - 1s 2ms/step - loss: 1013901376.0000 - val_loss: 1128879872.0000
Epoch 70/100
583/583 [==============================] - 1s 2ms/step - loss: 1013485760.0000 - val_loss: 1138415872.0000
Epoch 71/100
583/583 [==============================] - 1s 1ms/step - loss: 1012424832.0000 - val_loss: 1146824832.0000
Epoch 72/100
583/583 [==============================] - 1s 3ms/step - loss: 1010913984.0000 - val_loss: 1156554752.0000
Epoch 73/100
583/583 [==============================] - 1s 2ms/step - loss: 1010836800.0000 - val_loss: 1149724416.0000
Epoch 74/100
583/583 [==============================] - 1s 2ms/step - loss: 1010277760.0000 - val_loss: 1132092544.0000
Epoch 75/100
583/583 [==============================] - 1s 2ms/step - loss: 1008625856.0000 - val_loss: 1136130048.0000
Epoch 76/100
583/583 [==============================] - 1s 2ms/step - loss: 1007546176.0000 - val_loss: 1139191296.0000
Epoch 77/100
583/583 [==============================]

In [5]:
# Make predictions using the trained model and see the differences
y_pred_test = model.predict([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test]).flatten()
results_df = pd.DataFrame({'Actual Price': y_test, 'Predicted Price': y_pred_test})
results_df['Difference'] = results_df['Actual Price'] - results_df['Predicted Price']

print(results_df)

183/183 [==============================] - 0s 1ms/step
       Actual Price  Predicted Price    Difference
6345    41872.19512     48581.101562  -6708.906443
10578   88001.01927     91765.093750  -3764.074480
27972   58321.94495     29961.025391  28360.919559
11384  114573.90920    113057.335938   1516.573262
12272  112246.43210    131358.937500 -19112.505400
...             ...              ...           ...
1434    73490.78126     74188.757812   -697.976552
7663   272981.61430    250482.500000  22499.114300
14767  153616.64780    206271.484375 -52654.836575
27658   35530.24270     66413.960938 -30883.718237
27479   83775.38292     82043.234375   1732.148545

[5828 rows x 3 columns]


In [12]:
# Load test data
test_data_path = 'https://github.com/pedrogasparotti/challenges/raw/main/cars_test.csv'
df_test = load_data(test_data_path)

# Preprocess test data
X_numerical_test, X_marca_test, X_modelo_test, X_versao_test = preprocess_data(df_test)

# Make predictions on the test data
predictions = model.predict([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test])

# Add predictions to a new DataFrame
df_predictions = pd.DataFrame(predictions, columns=['predicted_price'])

# Concatenate test data with predictions DataFrame
df_result = pd.concat([df_test, df_predictions], axis=1)

# Save the combined DataFrame to a new CSV file
predictions_file_path = 'cars_test_predictions.csv'
df_result.to_csv(predictions_file_path, index=False)

print("Predictions saved to:", predictions_file_path)

309/309 [==============================] - 0s 1ms/step
Predictions saved to: cars_test_predictions.csv
